In [27]:
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.embed import components
from bokeh.palettes import Spectral11, Plasma256
from bokeh.transform import factor_cmap
from flask import Flask, render_template
from flask_bootstrap import Bootstrap
from db_objects import db
import pandas as pd
from flask import render_template
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.embed import components
from bokeh.resources import CDN
from db_objects import db
import pandas as pd
import sqlalchemy
import os
from bokeh.io import output_file, show
from bokeh.palettes import Spectral5, Spectral11, Spectral3, Spectral4, Spectral6
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg_clean as df
from bokeh.transform import factor_cmap

In [28]:

os.listdir()


['helper_functions.py',
 'profiler_stats.txt',
 'testing.py',
 '__pycache__',
 'readme.md',
 'profiler_stats_processer.txt',
 'FlaskApp',
 '.ipynb_checkpoints',
 'processers',
 'imobiliare',
 '.idea',
 'storia',
 'dags',
 'venv',
 'csv_files',
 'notebook_csv.ipynb',
 'line_profiler',
 'db_objects']

In [29]:
db = db.DbRealEstate()

In [30]:
df = pd.read_sql_table('scrapped_real_estate', db.engine)
df

,header_id,num_of_rooms,surface,compartment,floor,construction_year,property_type,parking_space,balcony,location_city,location_area,price_value,price_currency,source,seller_type,header_date
0,5459126,2.0,50.0,semidecomandat,3,1980.0,apartament,None,None,Cluj (judet),Strada Horea,350.0,€,www.storia.ro,None,2021-04-04
1,5459062,2.0,57.0,semidecomandat,3,2019.0,apartament,Da,Da,Cluj (judet),Marasti,380.0,€,www.storia.ro,None,2021-04-04
2,5459217,2.0,60.0,semidecomandat,5,2018.0,apartament,Da,Da,Cluj (judet),Zorilor,450.0,€,www.storia.ro,None,2021-04-04
3,5457702,4.0,135.0,decomandat,2,2010.0,apartament,None,None,Cluj (judet),Strada Episcop Marton Aron,620.0,€,www.storia.ro,None,2021-03-18
4,5459113,2.0,55.0,decomandat,2,1980.0,apartament,Da,Da,Cluj (judet),Marasti,380.0,€,www.storia.ro,None,2021-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46473,5382709,1.0,38.0,decomandat,4,1997.0,None,None,None,Brasov (judet),Brasov,550.0,RON,www.storia.ro,None,2021-06-26
46474,5702134,2.0,54.0,None,4,1997.0,None,None,None,Brasov (judet),Brasov,300.0,RON,www.storia.ro,None,2021-06-27
46475,5133834,2.0,56.0,decomandat,3,1990.0,None,None,None,Brasov (judet),Brasov,350.0,€,www.storia.ro,None,2021-06-23
46476,5710620,1.0,30.0,decomandat,2,NaN,garsoniera,None,None,Brasov (judet),Astra,200.0,€,www.storia.ro,None,2021-06-28


In [ ]:
    df = pd.read_sql_table('scrapped_real_estate', db.engine)
    df = df.dropna(axis=0, subset=['price_value'])
    df = df[~df.location_area.str.contains("Strada|Aleea|Bulevardul|Piata|P-ta")]
    df = df[~df.location_area.str.contains("Aleea")]
    df = df[~df.location_area.str.contains("Bulevardul")]
    df = df[~df.location_area.str.contains("Piata")]
    df = df[~df.location_area.str.contains("P-ta")]
    df.location_area = df.location_area.astype(str)
    df['location_area'] = df['location_area'].str.strip()
    df_cluj = df[df.location_city.str.contains('Cluj')]
    df_cluj = df[df.price_currency.str.contains('€')]
    group = df_cluj.groupby('location_area')
    source = ColumnDataSource(group)

    source_cmap = factor_cmap('location_area', palette=Plasma256, factors=sorted(df_cluj.location_area.unique()))

    p = figure(plot_height=650, plot_width=1200, x_range=group, title="Average price", toolbar_location=None, tools="")

    p.vbar(x='location_area', top='price_value_mean', width=1, source=source, line_color=source_cmap, fill_color=source_cmap)

    p.y_range.start = 0
    p.xgrid.grid_line_color = None
    p.xaxis.axis_label = "Location areas"
    p.xaxis.major_label_orientation = 1.2
    p.outline_line_color = None
    script5, div5 = components(p)
    cdn_js5 = CDN.js_files[0]
    show(p)

In [139]:
    df = pd.read_sql_table('scrapped_real_estate', db.engine)
    df = df.dropna(axis=0, subset=['price_value'])
    df = df[~df.location_area.str.contains("Strada|Aleea|Bulevardul|Piata|P-ta")]
    df.location_area = df.location_area.astype(str)
    df['location_area'] = df['location_area'].str.strip()
    df_cluj = df[df.location_city.str.contains('Cluj|cluj|CLUJ')]
    df_cluj = df[df.price_currency.str.contains('€|EUR')]
    df_cluj.loc[df_cluj['location_area'].str.contains('centru', case=False), 'location_city'] = 'Centru'
    #df_cluj = df[df.location_area.str.contains('Andrei Muresanu|Bulgaria|Buna Ziua|Centru|Manastur|Europa|gheorgheni|faget|grigorescu|gruia|iris|intre lacuri|marasti|someseni|sopor|zorilor|dambul rotund|becas|borhanci', case=False)]
    df_cluj = df_cluj['location_area'].str.contains(df_cluj['location_area'].value_counts()[:24])
    group = df_cluj.groupby('location_area')
    source = ColumnDataSource(group)

    source_cmap = factor_cmap('location_area', palette=Plasma256, factors=sorted(df_cluj.location_area.unique()))

    p = figure(plot_height=650, plot_width=1200, x_range=group, title="Average price", toolbar_location=None, tools="")

    p.vbar(x='location_area', top='price_value_mean', width=1, source=source, line_color=source_cmap, fill_color=source_cmap)

    p.y_range.start = 0
    p.xgrid.grid_line_color = None
    p.xaxis.axis_label = "Location areas (Cluj-Napoca)"
    p.xaxis.major_label_orientation = 1.2
    p.outline_line_color = None
    script5, div5 = components(p)
    cdn_js5 = CDN.js_files[0]
    show(p)

/home/dan/PycharmProjects/DissertationProject/venv/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [88]:
df = pd.read_sql_table('scrapped_real_estate', db.engine)
df = df.dropna(axis=0, subset=['header_date'])
df.loc[df['location_city'].str.contains('Cluj|cluj|CLUJ'), 'location_city'] = 'Cluj-Napoca'


df.location_city = df.location_city.astype(str)
df.header_date = pd.to_datetime(df['header_date'], format='%Y-%m-%d').dt.year
df.header_date = df.header_date.astype(str)

group = df.groupby('header_date')

index_cmap = factor_cmap('header_date', palette=Spectral5, factors=sorted(df.header_date.unique()), end=1)

p = figure(plot_width=1200, plot_height=650, title="Pretul mediu al anunturilor in timp",
           x_range=group, toolbar_location=None, tooltips=[("Price", "@price_value_mean"), ("location, date", "@location_city_header_date")])

p.vbar(x='header_date', top='price_value_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap, )

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Anul postarii anunturilor"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

show(p)

In [154]:
df = pd.read_sql_table('scrapped_real_estate', db.engine)
df = df.dropna(axis=0, subset=['header_date'])
df.loc[df['location_city'].str.contains('Cluj|cluj|CLUJ'), 'location_city'] = 'Cluj-Napoca'
df = df[df.num_of_rooms <= 4]

df.location_city = df.location_city.astype(str)
df.num_of_rooms = df.num_of_rooms.astype(str)
df.header_date = df.header_date.astype(str)

group = df.groupby(by=['location_city', 'num_of_rooms'])

index_cmap = factor_cmap('location_city_num_of_rooms', palette=Spectral5, factors=sorted(df.location_city.unique()), end=1)

p = figure(plot_width=1200, plot_height=650, title="Pretul mediu al anunturilor",
           x_range=group, toolbar_location=None, tooltips=[("Price", "@price_value_mean"), ("location, num_rooms", "@location_city_num_of_rooms")])

p.vbar(x='location_city_num_of_rooms', top='price_value_mean', width=1, source=group,
       line_color="white", fill_color=index_cmap)

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Numarul anunturilor per Oras"
p.xaxis.major_label_orientation = 1.2
p.outline_line_color = None

show(p)

In [42]:
df = pd.read_sql_table('scrapped_real_estate', db.engine)
df_cluj = df[df.location_city.str.contains('Cluj', case=False)]
df_cluj = df_cluj.dropna(axis=0, subset=['surface'])
df_cluj = df_cluj.dropna(axis=0, subset=['price_value'])
df_cluj.num_of_rooms = df_cluj.num_of_rooms.astype(str)
group = df_cluj.groupby('surface')
source = ColumnDataSource(group)

p = figure(plot_width=1200, plot_height=650)

p.circle('surface', 'price_value_mean', size=10, color="navy", alpha=0.5, source=source)
p.xaxis.axis_label = "Surface (square meters)"
p.yaxis.axis_label = "Average price"
script4, div4 = components(p)
cdn_js4 = CDN.js_files[0]
show(p)